In [1]:
# Dependencies
from census import Census
from config_sh import api_key
import hvplot.pandas
import pandas as pd
import requests
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings('ignore')

years = [2019, 2020, 2021, 2022]

In [5]:
# Initialize empty list to collect data
all_years_data = []

# List of Census variables to fetch
variables = [
    "B25003_002E", # Owner-occupied units
    "B25003A_002E", # Owner occupied (White alone)
    "B25003B_002E", # Owner occupied (Black or African American alone)
    "B25003C_002E", # Owner occupied (American Indian and Alaska Native alone)
    "B25003D_002E", # Owner occupied (Asian alone)
    "B25003E_002E", # Owner occupied (Native Hawaiian and Other Pacific Islander alone)
    "B25003F_002E", # Owner occupied (Some other race alone)
    "B25003G_002E", # Owner occupied (Two or more races)
    "B25003H_002E", # Owner occupied (White alone, not Hispanic or Latino)
    "B25003I_002E", # Owner occupied (Hispanic or Latino)
    "B19013A_001E", # Median Household Inc White alone
    "B19013B_001E", # Median Household Inc Black or African American alone
    "B19013C_001E", # Median Household Inc American Indian and Alaska Native alone
    "B19013D_001E", # Median Household Inc Asian alone
    "B19013E_001E", # Median Household Inc Native Hawaiian and Other Pacific Islander alone
    "B19013F_001E", # Median Household Inc Some other
    "B19013G_001E", # Median Household Inc Two or more
    "B19013H_001E", # Median Household Inc White alone, not Hisp
    "B19013I_001E", # Median Household Inc Hispanic or Latino
    "B25003_001E", # Total occupied units
    "B25064_001E", # Median gross rent
    "B25001_001E", # Total housing units
    "B19013_001E", # Median household income
    "B25088_002E", # Median monthly owner costs (with a mortgage)
    "B01003_001E", # Total population
    "B25006_002E", # White alone
    "B25006_003E", # Black or African American alone
    "B25006_004E", # American Indian and Alaska Native alone
    "B25006_005E", # Asian alone
    "B25006_006E", # Native Hawaiian and Other Pacific Islander alone
    "B25006_007E", # Some other race alone
    "B25006_008E", # Two or more races
    "B17021_002E"  # Count of individuals whose income in the past 12 months is below the poverty level
]


# Fetch data for each year
for year in years:
    c = Census(api_key, year=year)
    data = c.acs5.get(
        variables,
        {'for': 'zip code tabulation area:*'}
    )
    # Convert each year's data to DataFrame
    df_year = pd.DataFrame(data)
    df_year['year'] = year  # Add a column for the year
    all_years_data.append(df_year)

# Concatenate all data into a single DataFrame
census_data = pd.concat(all_years_data, ignore_index=True)

# Format zip code field
census_data['zip code tabulation area'] = census_data['zip code tabulation area'].astype(str).str.zfill(5)

census_data = census_data.rename(
    columns = {
        "B25003_002E": "Owner_Occupied_Units",
        "B25003A_002E": "Owner_White_alone",
        "B25003B_002E": "Owner_Black_African_American_alone",
        "B25003C_002E": "Owner_American_Indian_Alaska_Native_alone",
        "B25003D_002E": "Owner_Asian_alone",
        "B25003E_002E": "Owner_Native_Hawaiian_Other_Pacific_Islander_alone",
        "B25003F_002E": "Owner_Some_other_race_alone",
        "B25003G_002E": "Owner_Two_or_more_races",
        "B25003H_002E": "Owner_White_no_Hispanic_or_Latino",
        "B25003I_002E": "Owner_Hispanic_Latino",
        "B19013A_001E": "MHI White alone",
        "B19013B_001E": "MHI Black or African American alone",
        "B19013C_001E": "MHI American Indian and Alaska Native alone",
        "B19013D_001E": "MHI Asian alone",
        "B19013E_001E": "MHI Hawaiian and Other Pacific Islander alone",
        "B19013F_001E": "MHI Some other",
        "B19013G_001E": "MHI Two or more",
        "B19013H_001E": "MHI White alone, not Hisp",
        "B19013I_001E": "MHI Hispanic",
        "B25003_001E": "Total_Occupied_Units",
        "B25064_001E": "Median_Gross_Rent",
        "B25001_001E": "Total_Housing_Units",
        "B19013_001E": "Median_Household_Income",
        "B25088_002E": "Median_Monthly_Owner_Costs",
        "B01003_001E": "Total_Population",
        "B25006_002E": "White",
        "B25006_003E": "Black_African_American",
        "B25006_004E": "American_Indian_Alaska_Native",
        "B25006_005E": "Asian",
        "B25006_006E": "Native_Hawaiian_Other_Pacific_Islander",
        "B25006_007E": "Other_Race",
        "B25006_008E": "Two_More_Races",
        "B17021_002E": "Count_Inds_Income_Past_12_Mos_<_Pov_Level",
        "zip code tabulation area": "Zip_code"
    }
)
# Remove rows where any of the cells in that row is NaN
# census_data_clean = census_data.dropna(subset=['Median_Monthly_Owner_Costs'])

# First, remove rows with invalid values
census_data_clean = census_data[census_data['MHI White alone'] != -666666666]

# Then, drop rows with any NaNs in specified columns
ensus_data_clean = census_data_clean.dropna(subset=['MHI White alone'])


# Export the cleaned data to a CSV file
census_data_clean.to_csv("Resources/census_data_race_MHI.csv", encoding="utf-8", index=False)

# Display combined data
census_data_clean.head()



ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))